## Challenge Set 9: Part 2
#### *Intermediate level SQL* ####
Subject:      Baseball  
Date:         10/18/2018  
Name:         Steven Bierer  

----

### Login via terminal, if necessary
ssh -i ~/.ssh/id_rsa steve@52.12.221.98   

psql steve_metis  
\connect baseball

\d    (to see what tables are available)  
\q    (to quit)  

### Create the SSH tunnel and acess the data

In [60]:
# First the SSH tunneling #
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '34.222.59.241'
AWS_USERNAME = 'steve'
SSH_KEY_PATH = '/Users/neuromac/.ssh/id_rsa'    # use "ssh-keygen -p" to set password to nothing

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 51968


In [61]:
# Then create a new SQL engine linked to the BASEBALL database #
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost'
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'steve'
POSTGRES_PASSWORD = 'nathan08'
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
# cnx = create_engine(postgres_str).raw_connection()
cnx = create_engine(postgres_str)
print(f'Postgres SQL object type {type(cnx)} created.')

Postgres SQL object type <class 'sqlalchemy.engine.base.Engine'> created.


In [9]:
# Take a look at what tables are available in this database. #
# Some tables listed here were created later in the notebook.
import pandas as pd

query = ''' SELECT * FROM pg_catalog.pg_tables
            WHERE tableowner = 'steve';'''
pd.read_sql_query(query, cnx)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,allstarfull,steve,None,True,False,False,False
1,public,salaries,steve,None,True,False,False,False
2,public,schools,steve,None,True,False,False,False
3,public,franchises,steve,None,True,False,False,False
4,public,schoolsplayers,steve,None,False,False,False,False
5,public,fielding,steve,None,False,False,False,False


### Question 1 -- What was the total spent on salaries by each team, each year?
*__Answer__: See the query output below.*

In [7]:
# Order information alphabetically by team id, with latest year first #
query = ''' SELECT franchid, franchname, yearid, SUM(salary) AS "Total Salaries"
            FROM Salaries
            JOIN Franchises ON Salaries.teamid = Franchises.franchid
            GROUP BY franchid, yearid
            ORDER BY franchid ASC, yearid DESC
            LIMIT 15; '''
pd.read_sql_query(query, cnx)

,franchid,franchname,yearid,Total Salaries
0,ANA,Los Angeles Angels of Anaheim,2004,100534667.0
1,ANA,Los Angeles Angels of Anaheim,2003,79031667.0
2,ANA,Los Angeles Angels of Anaheim,2002,61721667.0
3,ANA,Los Angeles Angels of Anaheim,2001,47535167.0
4,ANA,Los Angeles Angels of Anaheim,2000,51464167.0
5,ANA,Los Angeles Angels of Anaheim,1999,55388166.0
6,ANA,Los Angeles Angels of Anaheim,1998,41281000.0
7,ANA,Los Angeles Angels of Anaheim,1997,31135472.0
8,ARI,Arizona Diamondbacks,2013,90132000.0
9,ARI,Arizona Diamondbacks,2012,73804833.0


### Question 2 -- What is the first and last year played for each player?
*__Answer__: Beginning with 1871, the players listed are shown in the query output below.*

First, in the PSGL terminal I entered the following lines. Note, the "cut" command forces only the first few columns to be taken from the .csv file.
```
CREATE TABLE IF NOT EXISTS Fielding (
            playerID varchar(10) NOT NULL,
            yearID int NOT NULL,
            stint int DEFAULT NULL
            );
COPY Fielding FROM PROGRAM 'cut -d "," -f 1,2,3 /home/steve/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;
```

In [16]:
# List out all players and their first and last year, in order of their first year played,  #
query = ''' SELECT playerid, MIN(yearid) AS "First Year", MAX(yearid) AS "Last Year"
            FROM Fielding
            GROUP BY playerid
            ORDER BY "First Year", playerid
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,playerid,First Year,Last Year
0,abercda01,1871,1871
1,addybo01,1871,1877
2,allisar01,1871,1876
3,allisdo01,1871,1883
4,ansonca01,1871,1897


### Question 3 -- Who has played the most all star games?
*__Answer__: Highest on the list is Hank Aaron, "AaronHA01", with 25 games. Note that from 1959 to 1962, two All-Star games were played per year (and Hank Aaron was there for all of them).*

In [4]:
# List out the players in order of number of A.S. games played. #
query = ''' SELECT playerid, COUNT(playerid) AS "Number of Games"
            FROM AllStarFull
            GROUP BY playerid
            ORDER BY "Number of Games" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,playerid,Number of Games
0,aaronha01,25
1,mayswi01,24
2,musiast01,24
3,mantlmi01,20
4,willite01,19


### Question 4 -- Which school has generated the most distinct players?
*__Answer__: University of Southern California (USC) has the highest number of distinct players, at 102.*

In [14]:
# Create and fill a new table from SchoolsPlayers.csv. #
# But this time, instead of executing the SQL commands from the command line, I will run them from Jupyter.
cmd_query = '''CREATE TABLE IF NOT EXISTS SchoolsPlayers (
            playerID varchar(10) NOT NULL,
            schoolID varchar(20) DEFAULT NULL,
            yearMin int DEFAULT NULL,
            yearMax int DEFAULT NULL
            ); '''
cur = cnx.raw_connection().cursor()
cur.execute(cmd_query)
cnx.commit()
# cnx.execute(cmd_query);

In [30]:
cmd_query = '''COPY SchoolsPlayers FROM '/home/steve/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;'''
cur = cnx.raw_connection().cursor()
cur.execute(cmd_query)
cnx.commit()

In [44]:
# Take a peek at the table #
query = ''' SELECT * FROM SchoolsPlayers
            LIMIT 5;'''
pd.read_sql_query(query, cnx)

,playerid,schoolid,yearmin,yearmax
0,aardsda01,pennst,2001,2001
1,aardsda01,rice,2002,2003
2,abbeybe01,vermont,1888,1892
3,abbotgl01,carkansas,1970,1970
4,abbotje01,kentucky,1991,1992


In [39]:
query = ''' SELECT SchoolID, COUNT(DISTINCT PlayerID) AS "Number of Players"
            FROM SchoolsPlayers
            GROUP BY SchoolID
            ORDER BY "Number of Players" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,schoolid,Number of Players
0,usc,102
1,texas,100
2,arizonast,98
3,stanford,82
4,michigan,77


### Question 5 -- Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. 
*__Answer__:  Nick Altrock had the longest career (among players with the correct information) at 35 years. He was followed by Jim O'Rourke, Minnie Minoso, and Charlie O'Leary, all of whom played for at least 30 years.*

In [25]:
# Create a new table from Master.csv. #
# As in Q4, I will create the table creation and filling from Jupyter.
cmd_query = '''CREATE TABLE IF NOT EXISTS Master (
            playerID varchar(10) NOT NULL,
            nameFirst varchar(30) DEFAULT NULL,
            nameLast varchar(30) DEFAULT NULL,
            debut date DEFAULT NULL,
            finalGame date DEFAULT NULL,
            retroID varchar(15) DEFAULT NULL,
            bbrefID varchar(15) DEFAULT NULL
            ); '''
cnx.execute(cmd_query);

In [26]:
# Double-check that it's in the table list #
query = ''' SELECT * FROM pg_catalog.pg_tables
            WHERE tableowner = 'steve';'''
pd.read_sql_query(query, cnx)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,allstarfull,steve,None,True,False,False,False
1,public,salaries,steve,None,True,False,False,False
2,public,schools,steve,None,True,False,False,False
3,public,franchises,steve,None,True,False,False,False
4,public,schoolsplayers,steve,None,False,False,False,False
5,public,master,steve,None,False,False,False,False
6,public,fielding,steve,None,False,False,False,False


In [30]:
# Fill the new table #
cmd_query = '''COPY Master FROM PROGRAM 'cut -d "," -f 1,14,15,21,22,23,24 /home/steve/baseballdata/Master.csv' ''' + \
            '''DELIMITER ',' CSV HEADER;'''
cur = cnx.raw_connection().cursor()
cur.execute(cmd_query)
cnx.raw_connection().commit()

In [47]:
# Use the command DATE_PART to get the difference in months #
query = ''' SELECT  PlayerID, NameFirst, NameLast, Debut, FinalGame,
                    (DATE_PART('year', FinalGame) - DATE_PART('year', Debut))*12
                    + DATE_PART('month', FinalGame) - DATE_PART('month', Debut)
                    AS "Career Span (Months)"
            FROM Master
            WHERE Debut IS NOT NULL AND FinalGame IS NOT NULL
            ORDER BY "Career Span (Months)" DESC
            LIMIT 5; '''
pd.read_sql_query(query, cnx)

,playerid,namefirst,namelast,debut,finalgame,Career Span (Months)
0,altroni01,Nick,Altrock,1898-07-14,1933-10-01,423.0
1,orourji01,Jim,O'Rourke,1872-04-26,1904-09-22,389.0
2,minosmi01,Minnie,Minoso,1949-04-19,1980-10-05,378.0
3,olearch01,Charley,O'Leary,1904-04-14,1934-09-30,365.0
4,lathaar01,Arlie,Latham,1880-07-05,1909-09-30,350.0


### Question 6 -- What is the distribution of debut months? 
*__Answer__: See the table output below. September features the most player debuts. I'm speculating that's the month when baseball teams mount one last effort to bring in new players during a bad season.*

In [53]:
# Use DATE_PART again #
query = ''' SELECT  DATE_PART('month', Debut) AS "Debut Month",
                    COUNT(DATE_PART('month', Debut))
            FROM Master
            WHERE Debut IS NOT NULL
            GROUP BY "Debut Month"
            ORDER BY "Debut Month"; '''
pd.read_sql_query(query, cnx)

,Debut Month,count
0,3.0,41
1,4.0,4711
2,5.0,2230
3,6.0,1893
4,7.0,1978
5,8.0,1943
6,9.0,5061
7,10.0,308


### Question 7 --  What is the effect of table join order on mean salary for the players listed in the main (master) table?
*__Answer__: The mean salary is 1.86 million dollars. There is no difference of join order, regardless of how the two tables are joined. However, I ran the same average on __just the Salaries table alone__ and obtained the same answer. It turns out that there are much fewer unique players in Salaries than in Master. So, as long as Master contains all the players that Salaries does, then this consistent answer across table joins makes sense.*

*I am assuming here that "mean salary" refers to the total mean across ALL players and ALL years.*

In [112]:
# First a LEFT JOIN with Master as the left table #
# Note: I'll join on Master.playerID as this has more similarity with Salaries.playerID than Master.bbrefID. 
query = ''' SELECT AVG(Salary), COUNT(Salary)
            FROM Master
            LEFT JOIN Salaries ON Master.playerID = Salaries.PlayerID;
            '''
pd.read_sql_query(query, cnx)

,avg,count
0,1.864357e+06,23956


In [118]:
# Then a RIGHT JOIN with Master as the right table #
query = ''' SELECT AVG(Salary), COUNT(Salary)
            FROM Salaries
            RIGHT JOIN Master ON Master.playerID = Salaries.PlayerID;
            '''
pd.read_sql_query(query, cnx)

,avg,count
0,1.864357e+06,23956


In [121]:
# And finally a LEFT JOIN with Salaries as the left table #
query = ''' SELECT AVG(Salary), COUNT(Salary)
            FROM Salaries
            LEFT JOIN Master ON Master.playerID = Salaries.PlayerID;
            '''
pd.read_sql_query(query, cnx)

,avg,count
0,1.864357e+06,23956


In [130]:
# Hmmm. For sanity, check the salaries for just the players in the Salaries table #
query = ''' SELECT AVG(Salary), COUNT(Salary)
            FROM Salaries;
            '''
pd.read_sql_query(query, cnx)

,avg,count
0,1.864357e+06,23956


2018-11-01 17:33:31,573| ERROR   | Socket exception: Connection reset by peer (54)


In [129]:
# Show the number of unique players in the Salaries table #
query = ''' SELECT COUNT(DISTINCT PlayerID)
            FROM Salaries;
            '''
pd.read_sql_query(query, cnx)

,count
0,4668


In [128]:
# Also show the number of unique players in the Master table #
query = ''' SELECT COUNT(DISTINCT PlayerID)
            FROM Master;
            '''
pd.read_sql_query(query, cnx)

,count
0,18354
